# Goal of project
- Author: Arun Nemani
- Effectively extract features from all 20 categories of the 20-newsgroups dataset
- Train and fit a classification model to predict text inputs based on the extracted features
- Report the accuracies for each classification models

### Cloned repos as baseline
- https://github.com/stefansavev/demos/blob/master/text-categorization/20ng/20ng.py
- https://nlpforhackers.io/text-classification/


In [44]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import nltk
import string
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.grid_search import GridSearchCV

## Initialize and load dataset

First we split the dataset into a training, development, and testing set.
The purpose of splitting up the dataset in this manner is to ensure we do not bias or overfit our model by iteratively refining our model on the train and test sets. Instead, we fine tune our model on the train and development set, and invoke the test only once as a final input on the tuned model.

Thus, the entire dataset is split into the training set (70%), development set (15%), and test set (15%).

In [45]:
# Initializations
nltk.download('punkt')
categories = None #Indicating all 20 categories are included in dataset
remove = ('headers', 'footers', 'quotes') #Required to remove false features that result in overfitting
RANDOM_STATE = 35

# Load dataset
print("Loading 20 newsgroups dataset for categories:")
newsdata = fetch_20newsgroups(subset='all')
X_train, X_intermediate, Y_train, Y_intermediate = train_test_split(newsdata.data, newsdata.target, test_size=0.30, random_state=RANDOM_STATE)
X_dev, X_test, Y_dev, Y_test = train_test_split(X_intermediate, Y_intermediate, test_size=0.50, random_state=RANDOM_STATE)
print('Data loaded')
print()
print('Training data documents:', len(X_train))
print('Development data documents:', len(X_dev))
print('Test data documents:', len(X_test))
print()
print('Total Newsgroups :', newsdata.target_names)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/arunnemani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Loading 20 newsgroups dataset for categories:
Data loaded

Training data documents: 13192
Development data documents: 2827
Test data documents: 2827

Total Newsgroups : ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


## Feature Extractor 1

We start by implementing a simple CountVectorizer to extract features without any preprocessing or parameter tuning.

In [62]:
FE1 = CountVectorizer(analyzer= 'word')
Vocab = FE1.fit_transform(X_train)
print('FE1 vocabulary size is {} in {} documents'.format(Vocab.shape[1], Vocab.shape[0]))

FE1 vocabulary size is 142559 in 13192 documents


## Feature Extractor 2

FE1 shows a total vocabulary size of 142559 unique features (words) in 13192 documents.
However, there are lots of words in our feature extraction that SHOULD not be considered features (articles, punctuations, etc). This is where we need to incorporate three text preprocessing schemes.
- Stopwords: This technique allows the systematic removal of english words that are not unique to a feature or document ('the', 'and', 'is', '.', '?', etc)
- Stemming: This technique truncates words into their respective unique root stems. Ex (Flying, flown, flyer, all have the same root)
- Tokenization: This process systematically breaks up a corpus or string line into uniquely different words or sentences

In [6]:
def Stem_tokenize(text):
    stemmer = PorterStemmer()
    return [stemmer.stem(w) for w in word_tokenize(text)]

In [65]:
FE2 = CountVectorizer(analyzer= 'word', tokenizer=Stem_tokenize,
                                stop_words=stopwords.words('english') + list(string.punctuation))
Vocab = FE2.fit_transform(X_train)
print('FE2 vocabulary size is {} in {} documents'.format(Vocab.shape[1], Vocab.shape[0]))

FE2 vocabulary size is 181542 in 13192 documents


## Feature Extractor 3

FE2 shows a total vocabulary size of 181542 unique features (words) in 13192 documents.
Note that the vocabulary feature size increased from FE2 to FE3, however, FE3 is significantly more robust in classifying unique words due to our preprocessing (accuracy / sanity checks to follow).

Now that we have incorporated tokenization, stopwords (with punctuations), and stemming, we can fine tune our feature extraction method. We will remove all accents in our corpus and add bi-grams to our model to ensure that multi-word features are also being classified together. For ex. word such as "24 bit" or "image processing" are considered one feature and is a bi-gram.

In [67]:
FE3 = CountVectorizer(analyzer= 'word', tokenizer=Stem_tokenize,
                                stop_words=stopwords.words('english') + list(string.punctuation),
                                lowercase=True, strip_accents='ascii', ngram_range=(1,2))
Vocab = FE3.fit_transform(X_train)
print('FE3 vocabulary size is {} in {} documents'.format(Vocab.shape[1], Vocab.shape[0]))

FE3 vocabulary size is 1392137 in 13192 documents


## Feature Extractor 4

FE3 shows a total vocabulary size of 1392137 unique features (words) in 13192 documents.

Up till now, we have used a standard CountVectorizer that counts the number of unique features within a dataset. This can be problematic as the frequency of these features are not accounted for. This presents an overfitting challenge particularly for linear approaches.

Thus we utilize another feature extractor (TfidfVectorizer) that accounts for the frequency of uniquely identified features in our model. All of the tuning parameters can still be used.

In [68]:
FE4 = TfidfVectorizer(analyzer= 'word', tokenizer=Stem_tokenize,
                                stop_words=stopwords.words('english') + list(string.punctuation),
                                lowercase=True, strip_accents='ascii', ngram_range=(1,2))
Vocab = FE4.fit_transform(X_train)
print('FE4 vocabulary size is {} in {} documents'.format(Vocab.shape[1], Vocab.shape[0]))

FE4 vocabulary size is 1392137 in 13192 documents


## Feature Extractor 5

Now that we implemented a more robust feature extractor, we need a way to deal with very unique features that may overfit our models. These include features that may be typos, slang, or words used by a very small subset of documents. It is expected that our model accuracy will reduce, however, this approach will prevent overfitting.

This is achieved by setting the max_df and min_df parameters as below:
- min_df = 5: Remove features that occur in 5 or less documents
- max_df = 0.75: Remove features that appear in more than 75% of all documents. These may include common formatting chars or post script symbols part of the document.

In [7]:
FE5 = TfidfVectorizer(analyzer= 'word', tokenizer=Stem_tokenize,
                                stop_words=stopwords.words('english') + list(string.punctuation),
                                lowercase=True, strip_accents='ascii', ngram_range=(1,2),
                                min_df=5, max_df= 0.75)
Vocab = FE5.fit_transform(X_train)
print('FE5 vocabulary size is {} in {} documents'.format(Vocab.shape[1], Vocab.shape[0]))

FE5 vocabulary size is 85540 in 13192 documents


## Finalized Feature Extractor

Note that we have SIGNIFICANTLY reduced the number of vocabulary features in our vectorizer from 1392137 to 85540.
This is primarily due to the min_df parameter since changing the max_df does not impact the feature size significantly.
Basically, our previous vectorizers were overfitting features that were very sparse in the dataset.

FE5 will be our finalized feature extractor for this project.
Next we explore classification schemes.

## Feature Classification

First we need to understand the dataset before selection a classification model.
The matrix output of the feature extraction methods are very sparse with a small set of non-zero values. 

It is important to note that we will fine tune our classification models on the dev set USING the feature extraction model created on the training set.

Thus we will try Multinomial Naive-Bayes, regularized Logistic regression, and Stochastic Gradient Descent.

In [46]:
FE5 = TfidfVectorizer(analyzer= 'word', tokenizer= Stem_tokenize,
                                stop_words=stopwords.words('english') + list(string.punctuation),
                                lowercase=True, strip_accents='ascii', ngram_range=(1,2),
                                min_df=5, max_df= 0.75)
Vocab_train = FE5.fit_transform(X_train)
Vocab_dev = FE5.transform(X_dev)
print('FE5 training set vocabulary size is {} in {} documents'.format(Vocab_train.shape[1], Vocab_train.shape[0]))
print('FE5 dev set vocabulary size is {} in {} documents'.format(Vocab_dev.shape[1], Vocab_dev.shape[0]))

FE5 training set vocabulary size is 85540 in 13192 documents
FE5 dev set vocabulary size is 85540 in 2827 documents


In [47]:
classifier_nb = MultinomialNB()
params = {'alpha':[0.00001, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0]}
grid_classifier_nb = GridSearchCV(classifier_nb, params, scoring = 'accuracy')
grid_classifier_nb.fit(Vocab_train, Y_train)
pred = grid_classifier_nb.predict(Vocab_dev)
print("Multinomial NB optimal alpha: {}".format(grid_classifier_nb.best_params_))

classifier_lreg = LogisticRegression(penalty = 'l2', solver='sag', random_state=RANDOM_STATE, n_jobs=-1)
params = {'C':[0.0001, 0.001, 0.01, 0.1, 0.25, 0.5, 1.0, 2.0, 5.0]}
grid_classifier_lreg = GridSearchCV(classifier_lreg, params, scoring = 'accuracy')
grid_classifier_lreg.fit(Vocab_train, Y_train)
pred = grid_classifier_lreg.predict(Vocab_dev)
print("Logistic Regression optimal C: {}".format(grid_classifier_lreg.best_params_))

classifier_SGD = SGDClassifier(tol=0.0001, penalty="l2", random_state=RANDOM_STATE, n_jobs=-1)
params = {'alpha':[0.00001, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0]}
grid_classifier_SGD = GridSearchCV(classifier_SGD, params, scoring = 'accuracy')
grid_classifier_SGD.fit(Vocab_train, Y_train)
pred = grid_classifier_SGD.predict(Vocab_dev)
print("Stochastic Gradient Descent optimal alpha: {}".format(grid_classifier_SGD.best_params_))

Multinomial NB optimal alpha: {'alpha': 0.01}
Logistic Regression optimal C: {'C': 5.0}
Stochastic Gradient Descent optimal alpha: {'alpha': 0.0001}


## Optimal classification model

Now that we have identified the optimal parameters for each model. We will calculate the final accuracies on the test set and select the final classification model for this project.

Note that we predefine the regularization parameters for SGD and logistic regression classifiers.

The idea of regularization is to avoid learning very large weights, which are likely to fit the training data but do not generalize well. L2 regularization adds a penalty to the sum of the squared weights whereas L1 regularization computes add the penalty via the sum of the absolute values of the weights. The result is that L2 regularization makes all the weights relatively small, and L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

In this particular application, there are a number of features that are very sparse but unique in identifying newsgroups.
Thus, only "L2" regularization has been selected for all classification models.


In [49]:
Vocab_test = FE5.transform(X_test)
classifier_NB = MultinomialNB(alpha=0.01)
classifier_NB.fit(Vocab_train, Y_train)
pred = classifier_NB.predict(Vocab_test)
print("NB classifier accuracy: {}".format(round(metrics.accuracy_score(Y_test, pred),4)))

classifier_lreg = LogisticRegression(penalty = 'l2', solver='sag', C=5, random_state=RANDOM_STATE, n_jobs=-1)
classifier_lreg.fit(Vocab_train, Y_train)
pred = classifier_lreg.predict(Vocab_test)
print("Logistic regression classifier accuracy: {}".format(round(metrics.accuracy_score(Y_test, pred),4)))

classifier_SGD = SGDClassifier(tol=0.0001, penalty="l2", alpha=0.0001, random_state=RANDOM_STATE, n_jobs=-1)
classifier_SGD.fit(Vocab_train, Y_train)
pred = classifier_SGD.predict(Vocab_test)
print("Stochastic Gradient Descent classifier accuracy: {}".format(round(metrics.accuracy_score(Y_test, pred),4)))

NB classifier accuracy: 0.9073
Logistic regression classifier accuracy: 0.9059
Stochastic Gradient Descent classifier accuracy: 0.9165


## Finalized predictive model

Based on our preprocessing, parameter tuning, and model selection, a stochastic gradient descent classifier can accurately predict an input corpus into one of the 20 newsgroups. However, multinomial NB has marginally lower accuracy but perfoms significantly much faster than the other classifiers. 

Below we apply some sanity checks to see our prediction work in real-time.

In [52]:
def predictNewsGroup(text, clf):
    Vocab_test = FE5.transform([text])
    targets = newsdata.target_names
    idx = clf.predict(Vocab_test)
    print("Predicted newsgroup: {}".format(targets[int(idx)]))
    return

print("20 newsgroups: {}".format(newsdata.target_names))
print()
predictNewsGroup("A Honda CBR is a dope bike", classifier_SGD)
predictNewsGroup("NLP scripts are ownage", classifier_SGD)
predictNewsGroup("He is #1 player with the highest contract signed for the Minnesota Wild", classifier_SGD)
predictNewsGroup("I'll sell my Gamecube for $1000", classifier_SGD)
predictNewsGroup("Homs is really unstable right now", classifier_SGD)

20 newsgroups: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']

Predicted newsgroup: rec.motorcycles
Predicted newsgroup: comp.windows.x
Predicted newsgroup: rec.sport.hockey
Predicted newsgroup: misc.forsale
Predicted newsgroup: talk.politics.guns


# Closing remarks

While we developed a fairly accurate classifier to predict the news category based on an input corpus, there are several approaches for further model refinement outlined below:

- 
### Alternative classifiers
I specificaly chose Stochastic Gradient Descent classifier since this approach assumes conditional independence of the features (words). There are several approaches other approaches (such, support vector machines, ensemble methods, Random Forest, etc) that can also be tuned and applied. However, for the purposes of this project, the focus is on the feature extraction methods.
- 
### Optimization
Several oppurtunities for optimization. Namely, the nasty for loop for stemming. There is limited stemming support in feature extractors thus I had to resort to making my own implementation.
- 
### Feature extraction
Lots of opportunities to fine tune feature extraction that include tuning of max_df and min_df parameters, custom corpus filtering functions to remove web scrapping metadata, etc.

- 
### Alternative model performance metrics
We are currently reporting model performane using accuracy, but this may be improved by utilizing other metrics such as f-score, confusion matrices, CV error, and the BLEU score (specific to NLP problems)